In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [3]:
source('0_parameters.sh')

In [4]:
f <- '/oak/stanford/groups/mrivas/projects/PRS/private_output/202009_batch/snpnet.eval.2_refit.tsv'
lst_f <- 'GBE_ID.lst'
cat_f <- '/oak/stanford/groups/mrivas/users/ytanigaw/repos/rivas-lab/ukbb-tools/05_gbe/extras/20200812_GBE_category/GBE_category.20201024.tsv'

# output

summary_f <- '/oak/stanford/groups/mrivas/projects/PRS/GBE_data/traits.tsv'


In [5]:
lst_f %>% fread(head=F) %>% pull() -> lst


In [6]:
cat_f %>% 
fread(
    colClasses = c('#GBE_category'='character', 'GBE_ID'='character'),
    select=c('#GBE_category', 'GBE_ID', 'GBE_short_name')
) %>%
rename('GBE_category'='#GBE_category') %>%
rename('trait_name'='GBE_short_name', 'trait_category'='GBE_category') -> cat_df


In [15]:
file.path(res_d, PRS_pval_f) %>% fread() %>%
rename('phe'='#phe') -> PRS_pval_df


In [16]:
setdiff(
lst, PRS_pval_df %>% filter(variable == 'PRS') %>% pull(phe) 
)

character(0)

In [23]:
f %>%
fread(
    colClasses = c('#GBE_ID'='character'),
    select = c(
        '#GBE_ID', 'split', 'geno', 'covar', 'geno_covar', 'geno_delta', 'n_variables', 'family'
    )
) %>%
rename('GBE_ID'='#GBE_ID') %>%
filter(split == 'test', GBE_ID %in% lst) %>% 
unique() %>% select(-split) %>%
left_join(cat_df, by='GBE_ID') %>%
rename('trait'='GBE_ID') %>%
left_join(
    PRS_pval_df %>% filter(variable == 'PRS') %>%
    select(phe, P) %>% rename('WB_test_P' = 'P'),
    by=c('trait'='phe')
) %>%
select(
    trait_category, trait, trait_name, family,
    geno, covar, geno_covar, geno_delta, n_variables, WB_test_P
) %>% arrange(-geno_delta) -> df 


In [24]:
length(lst)

[1] 1772

In [25]:
df %>% dim()


[1] 1772   10

In [26]:
df %>% head()

trait_category,trait,trait_name,family,geno,covar,geno_covar,geno_delta,n_variables,WB_test_P
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>
Blood_assays,INI30790,Lipoprotein A,gaussian,0.5654911,2.867887e-04,0.5679085,0.5676217,2861,0.000000e+00
Biomarkers,INI10030790,Lipoprotein A,gaussian,0.5167727,1.715521e-05,0.5178684,0.5178512,7148,6.274963e-51
Lifestyle_and_environment,BIN_FC2001747,"Hair colour (natural, before greying) red",binomial,0.9606803,5.563726e-01,0.9613973,0.4050248,1621,0.000000e+00
Blood_assays,INI30100,Mean platelet volume,gaussian,0.3602352,1.785971e-03,0.3619998,0.3602139,24114,0.000000e+00
Biomarkers,INI10030840,Total bilirubin,gaussian,0.3560721,1.627515e-05,0.3562139,0.3561976,544,2.104588e-145
Lifestyle_and_environment,BIN_FC1001747,"Hair colour (natural, before greying) blonde",binomial,0.8475395,5.314784e-01,0.8493170,0.3178386,6970,0.000000e+00


In [27]:
df %>%
rename('#trait_category' = 'trait_category') %>%
fwrite(summary_f, sep='\t', na = "NA", quote=F)


In [28]:
summary_f

[1] "/oak/stanford/groups/mrivas/projects/PRS/GBE_data/traits.tsv"